In [20]:
from bs4 import BeautifulSoup
from dbFunctions import as_sql, products_with, products_without
import requests
import csv

product = input("what do you want to find? ")
product = product.replace(" ", "+")
listing = []
custom_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36', 
    'Accept-language': 'en-US,en;q=0.7'
}

In [21]:
#opening amazon and grabbing all the items
amz = 'https://www.amazon.ca/s?k=' + product
res = requests.get(amz, headers= custom_header)
soup = BeautifulSoup(res.content, "html.parser")
print(amz)

https://www.amazon.ca/s?k=iem


In [22]:
#filtering functions, used before exporting
def cut(items, src):
    for i in range(len(items)):
        src.remove(items[i])

def remove_out_of_stock(catalogue):
    removal = []
    for i in range(len(catalogue)):
        if type(catalogue[i]["price"]) == str:
            removal.append(catalogue[i])
    cut(removal, catalogue)

def remove_duplicate_items(catalogue):
    items = []
    removal = []
    for i in range(len(catalogue)):
        price = catalogue[i]["price"]
        name = catalogue[i]["name"]
        reviews = catalogue[i]["reviews"]
        rating = catalogue[i]["rating"]
        if {"price": price, "name": name, "rating": rating, "reviews": reviews} in items:
            removal.append(catalogue[i])
        else:
            items.append({"price": price, "name": name, "rating": rating, "reviews": reviews})
    cut(removal, catalogue)


In [23]:
#Grabs info straight from search for amazon
def scrape_page(page):
    items = page.find_all("div", attrs={"data-component-type": "s-search-result"})
    for prod in items:
        link = "https://www.amazon.ca" + prod.find("a", attrs={"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}).get("href")
        name = prod.find("span", attrs={"class": "a-size-base-plus a-color-base a-text-normal"}).text.strip()
        if prod.find("span", attrs={"class": "a-icon-alt"}) is None:
            rating = "No rating"
            reviews = "No reviews"
        else:
            rating = prod.find("span", attrs={"class": "a-icon-alt"}).text.strip()
            rating = float(rating[:rating.find(" ")])
            reviews = prod.find("span", attrs={"class": "a-size-base s-underline-text"}).text.strip()
            reviews = int(reviews.replace(",", ""))
        if prod.find("span", attrs={"class": "a-offscreen"}) is None:
            price = "Out of stock"
        else:
            price = (prod.find("span", attrs={"class": "a-offscreen"}).text.strip()[1:])
            price = float(price.replace(",", ""))
        listing.append({"price": price, "rating": rating, "reviews": reviews,"name": name, "link": link})


In [24]:
#Choose filters
def filters(catalogue) -> str:
    cmd = input("Remove items out of stock?")
    if cmd.lower() in ["yea", "yes", "ye", "ya", "y"]:
        remove_out_of_stock(catalogue)
    
    cmd = input("Do you want the data as a CSV, SQL db or both?")
    return cmd.lower()

In [25]:
#Export data as CSV, for excel users
def as_csv(catalogue):
    with open("data.csv", 'w', newline='', encoding="utf-8") as csvf:
        fields = ["price", "rating", "reviews", "name", "link"]
        writer = csv.DictWriter(csvf, fieldnames= fields)
        writer.writeheader()
        writer.writerows(catalogue)

In [26]:
#Iterating through pages, add all items to the list, not removing duplicates or items out of stock
scrape_page(soup)
page_num = "2"
current = soup.find("a", attrs={"aria-label": "Go to next page, page " + page_num})
while current is not None:
    req = requests.get("https://www.amazon.ca" + current.get("href"), headers= custom_header)
    print("https://www.amazon.ca" + current.get("href"))
    sauce = BeautifulSoup(req.content, "html.parser")
    scrape_page(sauce)
    page_num = str(int(page_num) + 1)
    current = sauce.find("a", attrs={"aria-label": "Go to next page, page " + page_num})

https://www.amazon.ca/s?k=iem&page=2&qid=1731633615&ref=sr_pg_1
https://www.amazon.ca/s?k=iem&page=3&qid=1731633616&ref=sr_pg_2
https://www.amazon.ca/s?k=iem&page=4&qid=1731633618&ref=sr_pg_3
https://www.amazon.ca/s?k=iem&page=5&qid=1731633619&ref=sr_pg_4
https://www.amazon.ca/s?k=iem&page=6&qid=1731633620&ref=sr_pg_5
https://www.amazon.ca/s?k=iem&page=7&qid=1731633622&ref=sr_pg_6


In [27]:
#Exporting the data, asks for what filters they want, as .csv or .db
remove_duplicate_items(listing) #always remove dupes cuz yea
output = filters(listing)
if output == "both":
    as_sql(listing)
    as_csv(listing)
elif output == "csv":
    as_csv(listing)
else:
    as_sql(listing)

Inserted: {'price': 65.99, 'rating': 4.3, 'reviews': 15031, 'name': 'Linsoul KZ ZS10 Pro, 4BA+1DD 5 Driver in Ear Monitor, HiFi Wired Earbuds, Gaming Earbuds, Hybrid IEM Earphones with Stainless Steel Faceplate, Recessed 2 Pin Detachable Cable (Without Mic, Black)', 'link': 'https://www.amazon.ca/sspa/click?ie=UTF8&spc=MTo4NDUxMjE0MjQ5MjQ3NTgyOjE3MzE2MzM2MTU6c3BfYXRmOjMwMDE2NzYxMzA3OTcwMjo6MDo6&url=%2FKZ-ZS10-Linsoul-Earphones-Detachable%2Fdp%2FB07QKYTGH9%2Fref%3Dsr_1_1_sspa%3Fdib%3DeyJ2IjoiMSJ9.F3X65Ocp45QjwZ5o822fwtcdcw79Cyrk8T18E41tXf_yzd655CvtxzD-OUtXqh3MJz3MN1jBmZJhccpKKfXrKKBuItVR1OYu-f-z1Kecn7QJEyeGiHzx1Tiof2mq7riN8k4zblalCB8hQDT9UA1aw5UNP3blX5qlPDHt0fv8x96ZgVKKZH3hk_9ICGxbjzS7hiak-TnrNqQiba91ZVrHMtkFqqj2Htc2rNzwjU_1j_biPu1kHonWjRi1iPvwPEzjBAveN_PEJ9MVcmFcCRBWDX3UELVE8yFgGnCSvImSIeQ.H8DEAvCjJDBzvfazaaNS-mtyOSSN5Q_2CQEa5g6Wm4c%26dib_tag%3Dse%26keywords%3Diem%26qid%3D1731633615%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1'}
Inserted: {'price': 339.99, 'rating': 4.

In [28]:
#if output != "csv":
cmd = "?"
while cmd != "0":
    cmd = input("Choose command:")
    if cmd == "1":
        cmd = input("Only items with:")
        print(products_with(cmd))
    if cmd == "2":
        cmd = input("Only items without:")
        print(products_without(cmd))
        

In [29]:
import os
print(os.listdir('/mnt/data'))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/mnt/data'